In [ ]:
%load_ext autoreload
%autoreload 2

# Create Snomed Phrase Matcher

1. Inits SnomedPhraseMatcher
2. Add a set of parent cuis
3. Check reload works

In [ ]:
import pickle
from pathlib import Path

import spacy

from discharge_summaries.snomed.lookup import SnomedLookup
from discharge_summaries.snomed.phrase_matcher import SnomedPhraseMatcher

In [ ]:
SNOMED_DIR = Path.cwd().parent / "data" / "snomed"
OUTPUT_PATH = SNOMED_DIR / "snomed_phrase_matcher_full.pkl"

## 1. Init Phrase Matcher

In [ ]:
snomed_lookup = SnomedLookup.load(SNOMED_DIR)
nlp = spacy.load("en_core_sci_lg", disable=["ner", "parser"])
nlp.add_pipe("lower_case_lemmas", last=True)
snomed_matcher = SnomedPhraseMatcher(nlp, use_child_cui_label=True)

## 2. Add Set of Parent Cuis

Takes ~7 mins

In [ ]:
root_cui = 138875005

In [ ]:
snomed_matcher.add_parent_cui(138875005, snomed_lookup)

In [ ]:
OUTPUT_PATH.write_bytes(pickle.dumps(snomed_matcher))

## 2. Check Reload works

In [ ]:
reload_snomed_matcher = pickle.loads(OUTPUT_PATH.read_bytes())

In [ ]:
for parent_cui in {362981000, 900000000000441003}:
    reload_snomed_matcher._phrase_matcher.remove(str(parent_cui))
    for child_cui in snomed_lookup.get_child_cuis(parent_cui):
        reload_snomed_matcher._phrase_matcher.remove(str(child_cui))

In [ ]:
for ent in reload_snomed_matcher(
    "The patient has a history of heart attacks and strokes. HIV positive. arf"
).ents:
    print(ent.text, ent.label_)